# Import packages

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_circles
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
import numpy as np
import os
import pandas as pd
import numpy.matlib
from numpy import random
import os
import math
import warnings
warnings.filterwarnings("ignore")

# IMBoost

In [8]:
def fit_boosting(X_maj, X_min, y_maj, y_min, n_estimators=10):
    n_samples_maj, n_features = X_maj.shape
    n_samples_min, n_features = X_min.shape
    D_maj = np.ones((n_samples_maj, ))/n_samples_maj
    D_min = np.ones((n_samples_min, ))/n_samples_min
    estimators = []

    for t in range(n_estimators):
        print(t)
        D_maj = D_maj / (np.sum(D_maj)+  np.sum(D_min) )                 
        D_min =D_min / (np.sum(D_maj)+  np.sum(D_min)  )
        
        #combine x1,x2; y1,y2; d1,d2
        X=np.vstack((X_maj,X_min))
        y=np.hstack((y_maj,y_min))
        D=np.hstack((D_maj,D_min))
        h = DecisionTreeClassifier(max_depth=5)  
        h.fit(X, y, sample_weight=D)        

        ypred_maj = h.predict(X_maj)
        ypred_min = h.predict(X_min)
        e_maj = 1 - accuracy_score(y_maj, ypred_maj)  
        a_maj = 0.5 * np.log((1 - e_maj) / e_maj)               
        e_min = 1 - accuracy_score(y_min, ypred_min)  
        a_min = 0.5 * np.log((1 - e_min) / e_min)
        
        m_maj = (y_maj == ypred_maj) * 1 + (y_maj != ypred_maj) * -1  
        m_min = (y_min == ypred_min) * 1 + (y_min != ypred_min) * -1 
        D_maj *= np.exp(-a_maj * m_maj)
        D_min *= np.exp(-a_min * m_min)
        '''for i in range(len(m)):
            if m[i]==-1:
                print(y[i] ,": ", D[i])  '''   
        estimators.append((a_maj+a_min, h))  

    return estimators

# Predict

In [9]:
def predict_boosting(X, estimators):
    pred = np.zeros((X.shape[0], ))

    for a, h in estimators:
        pred += a * h.predict(X)

    y = np.sign(pred)

    return y

# read dataset and split samples

In [10]:
#data_name='vehicle0.dat'
data_name='glass1.dat'
Size_data=846
C_maj=647
F_maj=18
C_min=199
F_min=18
head=23 # number of line without sample (header)

if data_name=='glass1.dat':
    Size_data=214
    C_maj=138
    F_maj=9
    C_min=76
    F_min=9
    head=14 # index of first data sample 
if data_name=='ecoli-0_vs_1.dat':
    Size_data=220
    C_maj=143
    F_maj=7
    C_min=77
    F_min=7
    head=12 # index of first data sample 
if data_name=='wisconsin.dat':
    Size_data=683
    C_maj=444
    F_maj=9
    C_min=239
    F_min=9
    head=14 # index of first data sample 
if data_name=='iris0.dat':
    Size_data=150
    C_maj=100
    F_maj=4
    C_min=50
    F_min=4
    head=14 # index of first data sample   

In [11]:
#features = np.zeros((846,18))
#labels = np.zeros((846,),dtype=np.uint8)
Size_data=174
C_maj=146
F_maj=2
C_min=28
F_min=2
head=0 # index of first data sample 
features_maj = np.zeros((C_maj,F_maj))
labels_maj = np.zeros((C_maj,),dtype=np.uint8)
features_min = np.zeros((C_min,F_min))
labels_min = np.zeros((C_min,),dtype=np.uint8)
f1=0
f2=0
with open('data_all.txt','r') as file:
    a = file.readlines()
    for i in range(Size_data):
        a[i+head] = a[i+head].replace(" ","")
        b = a[i+head].split(',')

        if b[F_maj].rfind("-1")!=(-1):
            b[F_maj] = -1
            features_min[f2,:] = np.array(b[:F_maj])
            labels_min[f2] = b[F_maj] 
            f2+=1
        elif b[F_maj].rfind("1")!=(-1):
            b[F_maj] = 1
            features_maj[f1,:] = np.array(b[:F_maj])
            labels_maj[f1] = b[F_maj] 
            f1+=1
Xtrn_maj, Xtst_maj, ytrn_maj, ytst_maj = train_test_split(features_maj, labels_maj, test_size=0.25, random_state=13)
Xtrn_min, Xtst_min, ytrn_min, ytst_min = train_test_split(features_min, labels_min, test_size=0.25, random_state=13)


# report Metrics

In [ ]:

dim=F_maj
features_maj=Xtrn_maj
features_min=Xtrn_min
IMratio = len(features_maj)/len(features_min)
[minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]=kNNenemyPoint(features_min,features_maj);
majlabel = 1;
minlabel = -1;
mindradious = [];
for i in range(len(features_min)):
    ilabel = minlabel;
    dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
    mindradious.append([i,features_min[i,:],dradius]);
    #mindradious.append([i,dradius]);
mindradious=np.array(mindradious)
data = np.copy(features_min)
[mindradious,mindist] = unionSphere(data,mindradious);

#oversampling
uncoveredpoint = list(set(mindradious[:,-1]));
print(uncoveredpoint)
numofsphere = [];
for i in range(len(uncoveredpoint)):
        j = uncoveredpoint[i];
        numofsphere.append(np.sum(mindradious[:,-1]==j))
numofsphere=np.array(numofsphere)
complexitymeasure = len(uncoveredpoint)/len(features_min)
finalmincenters = np.array(mindradious[uncoveredpoint,1].tolist());
finalminradius  = np.array(mindradious[uncoveredpoint,2].tolist());
complexitymeasure = 1;
oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
print(oversamplesize)
alpha = 1;
beta = 1;
pi = np.array(np.exp(-alpha*finalminradius));
pj = np.array(np.exp(-beta*numofsphere));
pi =np.array((pi+pj)/np.sum(pi+pj)) 

suboversampesizes = np.ceil(pi*oversamplesize)
generatedX = [];
for i in range(len(uncoveredpoint)):
        X = gnerandsphere(finalmincenters[i,:],dim,int(suboversampesizes[i]),finalminradius[i]); 
        generatedX.append([X]);



In [12]:

estimators = fit_boosting(Xtrn_maj,Xtrn_min, ytrn_maj, ytrn_min)
ypred_maj = predict_boosting(Xtst_maj, estimators)
ypred_min = predict_boosting(Xtst_min, estimators)
from sklearn.metrics import accuracy_score
#tst_err = 1 - accuracy_score(ytst, ypred)
#tst_err
print("negative: ", accuracy_score(ytst_maj, ypred_maj))
print("positive: ", accuracy_score(ytst_min, ypred_min))

0
1
2
3
4


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.metrics import accuracy_score
g_mean = 1.0
    #
X=np.vstack((Xtst_maj,Xtst_min))
y=np.hstack((ytst_maj ,ytst_min ))
ypred = predict_boosting(X, estimators)
for label in np.unique(y):
    idx = (y == label)
    g_mean *= accuracy_score(y[idx], ypred[idx])
    #
g_mean = np.sqrt(g_mean)
score = g_mean
print(score)

# Oversampling

In [3]:
def gnerandsphere(center,d,N,r):
    random_directions = random.randn(N,d);
    s=np.sqrt(np.array(np.sum(random_directions**2,axis=1)))
    random_directions = random_directions/np.matlib.repmat(s,d,1).T; 
    random_radii =r*random.rand(N,1);
    Y = random_directions* np.matlib.repmat(random_radii,1,d);
    Y = np.matlib.repmat(center,N,1)+Y;
    return Y

In [4]:
def unionSphere(data,dataradious):
    pd = L2_dist(data,data);
    pddist =np.copy(pd);
    temp=[(v,i) for i,v in enumerate(dataradious[:,-1])]
    temp.sort(reverse=True)
    [dummy,sortid]=zip(*temp)
    sortdataradious = dataradious[sortid,:]
    #print("sortdataradious",sortdataradious )
    flag =np.zeros((len(data),1))-1
    for i in range(len(sortdataradious)):
        if flag[i] == -1:
            flag[i] = sortdataradious[i,0];
            leftindex =[]
            for j in range(i+1,len(sortdataradious)):
                if flag[j]==-1:
                    leftindex.append(j)
            
            leftunhandledid = sortdataradious[leftindex,0];
            iindex=sortdataradious[i,0]
            leftdist=[]
            for k in range(len(leftunhandledid)):
                jindex=leftunhandledid[k]
                leftdist.append(pd[iindex,jindex])
            leftdist=np.array(leftdist)
            includedset =np.argwhere(leftdist <= sortdataradious[i,2]).tolist();
            if len(includedset)>0:
                for k in range(len(includedset)):
                    in_id =includedset[k][0]
                    flag[leftindex[in_id]] = flag[i];

    belongto =np.zeros((len(dataradious),1));
    belongto[sortdataradious[:,0].tolist()] = flag;
    dataradious=dataradious.tolist()
    for k in range(len(dataradious)):
        dataradious[k].append(int(belongto[k,0]))
    dataradious=np.array(dataradious)    
    return [dataradious,pddist]

In [5]:
def calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor):
    IR = len(majENNdist)/len(minENNdist);
    tha = 1.0/(1+np.exp(-(IR-1)));
    if ilabel == minlabel:
        nej = int(minenemykneigor[i][0]);
        dij = minENNdist[i][0];
        jlabel =  majlabel;
        ENN=int(majenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*tha;
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;
    else:
        nej = int(majenemykneigor[i][0]);
        dij = majENNdist[i][0];
        jlabel =  minlabel;
        ENN=int(minenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*(1-tha);
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;


In [6]:
def L2_dist(Min_data,Maj_data):
    x = Min_data;
    xsup = Maj_data;
    ps = np.dot(x,xsup.T); 
    [nps,pps]=ps.shape;
    normx =np.array([np.sum(x**2, axis=1)]).T;
    normxsup = np.sum(xsup**2, axis=1);
    A=np.matlib.repmat(normx,1,pps)
    B=np.matlib.repmat(normxsup.T,nps,1)
    ps = -2*ps + A + B; 
    E=np.sqrt(ps);
    return E
    


In [7]:
def kNNenemyPoint(Min_data,Maj_data):
    k_nNum = 1;
    NT = len(Min_data)
    NTNT = len(Maj_data)
    minENNdist=np.ones((NT,k_nNum))*math.inf;
    minenemykneigor=np.ones((NT,k_nNum))*math.inf;
    majENNdist= np.ones((NTNT,k_nNum))*math.inf;
    majenemykneigor =np.ones((NTNT,k_nNum))*math.inf;
    pd = L2_dist(Min_data,Maj_data);
    for i in range(NT):
        d=np.copy(pd[i,:])
        min_id=[]
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        minENNdist[i,:] = pd[i, min_id];
        minenemykneigor[i,:] =  min_id
    pd = pd.T
    majmindistmax = pd;
    for i in range(NTNT):   
        d=np.copy(pd[i,:])
        min_id=[];
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        majENNdist[i,:] = pd[i,min_id];
        majenemykneigor[i,:] = min_id
    return [minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]